In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")

print("Orders Data Shape:", orders.shape)
orders.head()


Orders Data Shape: (10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")

print("Users Data Shape:", users.shape)
users.head()


Users Data Shape: (3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
# Create SQLite database from SQL file
conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants Data Shape:", restaurants.shape)
restaurants.head()


Restaurants Data Shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)

print("After merging Orders & Users:", merged_df.shape)
merged_df.head()


After merging Orders & Users: (10000, 9)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [6]:
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

print("Final Dataset Shape:", final_df.shape)
final_df.head()


Final Dataset Shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print("Final dataset saved as final_food_delivery_dataset.csv")


Final dataset saved as final_food_delivery_dataset.csv


In [8]:
final_df.groupby("membership")["order_id"].count()


membership
Gold       4987
Regular    5013
Name: order_id, dtype: int64

In [9]:
final_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

In [10]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [12]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[0, 3.5, 4.0, 4.5, 5.0],
    labels=["0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]
)




In [13]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.quarter

final_df.groupby("quarter")["total_amount"].sum()


quarter
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64

In [14]:
gold_orders_count = final_df[final_df["membership"] == "Gold"]["order_id"].count()
gold_orders_count


np.int64(4987)

In [15]:
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [16]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [17]:
avg_order_value_gold = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(avg_order_value_gold, 2)


np.float64(797.15)

In [18]:
high_rating_orders = final_df[final_df["rating"] >= 4.5]["order_id"].count()
high_rating_orders


np.int64(3374)

In [19]:
# Find top revenue city among Gold members
top_city_gold = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# Count orders in that city (Gold members only)
orders_top_city_gold = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city_gold)
]["order_id"].count()

top_city_gold, orders_top_city_gold


('Chennai', np.int64(1337))

In [20]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [21]:
final_df.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [22]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()

user_spend[user_spend > 1000].count()


np.int64(2544)

In [23]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
    include_lowest=True
)

final_df.groupby("rating_range")["total_amount"].sum()


C:\Users\vrmhc\AppData\Local\Temp\ipykernel_22752\2637048083.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum()


rating_range
3.0–3.5    2136772.70
3.6–4.0    1717494.41
4.1–4.5    1960326.26
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

In [24]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [25]:
final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values(["restaurant_count", "total_revenue"], ascending=[True, False])


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [26]:
gold_percentage = (
    final_df[final_df["membership"] == "Gold"].shape[0]
    / final_df.shape[0]
) * 100

round(gold_percentage)


50

In [28]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [29]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.quarter

final_df.groupby("quarter")["total_amount"].sum()


quarter
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64